In [29]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np
from matplotlib import pyplot as plt

df_weekly = pd.read_csv('6_mo_weekly.csv', sep='\t')

train = df_weekly[df_weekly['Date'] <= '2020-10-15']
train = train.drop(['Date', 'Release', 'Y'],axis=1)
y_train = train['Week + 1']
X_train = train.drop('Week + 1', axis = 1)

test = df_weekly[df_weekly['Date'] > '2020-10-15']
test = test.drop(['Date', 'Release', 'Y'],axis=1)
y_test = test['Week + 1']
X_test = test.drop('Week + 1', axis=1)

        
#scaler = StandardScaler()
#train_t = scaler.fit_transform(X_train)
#val_t = scaler.transform(X_test)

#X_train = pd.DataFrame(train_t)
#X_test = pd.DataFrame(val_t)


train_rmse_list = []
train_r2_list = []
test_rmse_list = []
test_r2_list = []

alphas = [-10] #[2**k for k in range(-10,10)]
for alpha in alphas:
    linreg = Ridge(alpha = alpha, fit_intercept=True)
    linreg.fit(X_train, y_train)
#    print('intercept:',linreg.intercept_)
#    print('theta:',linreg.coef_)
        
    mean = np.mean(y_train)

    y_train_pred = linreg.predict(X_train)
    y_test_pred = linreg.predict(X_test)

    train_rmse = mean_squared_error(y_train, y_train_pred,squared = False)
    train_r2 = r2_score(y_train,y_train_pred)

    test_rmse = mean_squared_error(y_test, y_test_pred, squared = False)
    test_r2 = r2_score(y_test, y_test_pred)

    print('Alpha', alpha)
    print('train RMSE:',(train_rmse))
    print('train R2', train_r2)
    print('test RMSE:',(test_rmse))
    print('test R2', test_r2)
    train_rmse_list.append(train_rmse)

    train_r2_list.append(train_r2)
    test_rmse_list.append(test_rmse)
    test_r2_list.append(test_r2)

Alpha -10
train RMSE: 119662.16154608905
train R2 0.9596427474343802
test RMSE: 246239.39492140207
test R2 0.7275983828230643


In [48]:
import plotly.graph_objects as go
import pandas as pd
from datetime import timedelta
import dash
import dash_core_components as dcc
import dash_html_components as html
import numpy as np
import plotly.graph_objects as go
from model import train_model
#from database import fetch_all_bpa_as_df



COLORS = ['rgb(67,67,67)', 'rgb(115,115,115)', 'rgb(49,130,189)', 'rgb(189,189,189)']
def static_stacked_trend_graph(stack=False):
    """
    Returns scatter line plot of all power sources and power load.
    If `stack` is `True`, the 4 power sources are stacked together to show the overall power
    production.
    """
    i=0
    df = pd.read_csv('3_mo_weekly.csv', sep='\t')
    df['Date_dt'] = df['Date'].astype('datetime64[ns]')
    date = df['Date_dt'].iloc[-1]
    #movie_list = df[df['Date_dt'] == date].sort_values(by=['Weekly'], ascending=False)['Release'][0:4]
    
    df_test = df.groupby('Release').filter(lambda x : x['Release'].shape[0]>=2)
    movie_list = df_test[df_test['Date_dt'] == date].sort_values(by=['Weekly'], ascending=False)['Release']

    df = df[df['Release'] == movie_list.iloc[i]]
    if df is None:
        return go.Figure()
    
    x = df['Date_dt'] 
    
    predict_date = df['Date_dt'].iloc[-1]  + timedelta(days=7)
    
    y_pred = train_model().predict([df.drop(['Release', 'Date', 'Y', 'Week + 1', 'Date_dt'], axis=1).iloc[-1]])
    trends = df['google trends']/np.sum(df['google trends'])
    rev = df['Weekly']/np.sum(df['Weekly'])
    
    fig = go.Figure()
    
    print(predict_date, y_pred)
    
    fig.add_trace(go.Scatter(x=x, y=df['Weekly'], mode='lines', name='Gross',
                             line={'width': 2, 'color': 'orange'},
                             stackgroup='stack' if stack else None))
    fig.add_trace(go.Scatter(x=[predict_date], y=y_pred, mode='markers', name='Google Trends',
                             line={'width': 2, 'color': 'red'},
                             stackgroup='stack' if stack else None))
    fig.update_layout(yaxis=dict(range=[0, 1.2*df['Weekly'].max()]), xaxis=dict(range=[x.iloc[0], predict_date+timedelta(days=1)]))

    title = f'Weekly Revenue and Trends for {movie_list.iloc[i]}'
    if stack:
        title += ' [Stacked]'
    fig.update_layout(template='plotly_dark',
                      title=title,
                      plot_bgcolor='#23272c',
                      paper_bgcolor='#23272c',
                      yaxis_title='Normalized Rev and Trends',
                      xaxis_title='Date/Time')
    return fig
static_stacked_trend_graph()

2020-12-14 00:00:00 [2167080.73058371]


In [42]:
import plotly.graph_objects as go
import pandas as pd
from datetime import timedelta


COLORS = ['rgb(67,67,67)', 'rgb(115,115,115)', 'rgb(49,130,189)', 'rgb(189,189,189)']
def static_stacked_trend_graph(stack=False):
    """
    Returns scatter line plot of all power sources and power load.
    If `stack` is `True`, the 4 power sources are stacked together to show the overall power
    production.
    """
    i=1
#    df = pd.read_csv('1_year_weekly.csv', sep='\t')
#    df['Date_dt'] = df['Date'].astype('datetime64[ns]')
#    date = df['Date_dt'].iloc[-1]
#    movie_list = df[df['Date_dt'] == date].sort_values(by=['Weekly'], ascending=False)['Release'][0:10]
#    
#    df = df[df['Release'] == movie_list.iloc[i]]
#    if df is None:
#        return go.Figure()
    
    #x = df['Date_dt'] 
    #predict_date = df['Date_dt'].iloc[-1]  + timedelta(days=7)
    
    #linreg.predict(df.iloc[-1])
    
    
    df = pd.read_csv('3_mo_weekly.csv', sep='\t')
    df['Date_dt'] = df['Date'].astype('datetime64[ns]')
    date = df['Date_dt'].iloc[-1]

    
    fig = go.Figure()
    movie_list = df[df['Date_dt'] == date].sort_values(by=['Weekly'], ascending=False)['Release'][0:20]
    #movie = movie_list.iloc[0]
    for movie in movie_list:
        date = df['Date_dt'].iloc[-1]
       # print(movie, date)
        trend = df[(df['Release'] == movie) & (df['Date_dt'] == date)]['google trends'].iloc[0]
        rev = df[(df['Release'] == movie) & (df['Date_dt'] == date)]['Weekly'].iloc[0]
        fig.add_trace(go.Scatter(x=[trend], y=[rev], mode='markers', name=movie,
                         line={'width': 2, 'color': 'orange'},
                         stackgroup='stack' if stack else None))
        print(trend, rev)
    
    title = f'Revenue and trends'
    if stack:
        title += ' [Stacked]'
    fig.update_layout(template='plotly_dark',
                      title=title,
                      plot_bgcolor='#23272c',
                      paper_bgcolor='#23272c',
                      yaxis_title='MW',
                      xaxis_title='Date/Time')
    return fig
static_stacked_trend_graph()

2.81 3970375.0
0.215904 686390.0
8.211538461538463 477130.0
8.333333333333334 400000.0
0.208416 341629.0
1.48 320510.0
0.56576 245000.0
0.0 239000.0
0.9875478260869563 234140.0
0.0 196871.0
2.673684210526316 189000.0
2.16 184925.0
2.8125 160000.0
2.3684210526315788 136000.0
0.2834 134000.0
0.0 95360.0
0.0288288 61846.0
0.17846399999999998 57223.0
0.8858754098360654 33288.0
1.3581176470588234 26861.0


In [43]:
import plotly.graph_objects as go
import pandas as pd
from datetime import timedelta


COLORS = ['rgb(67,67,67)', 'rgb(115,115,115)', 'rgb(49,130,189)', 'rgb(189,189,189)']
c = ['red', 'blue', 'orange', 'white']
def static_stacked_trend_graph(stack=False):
    """
    Returns scatter line plot of all power sources and power load.
    If `stack` is `True`, the 4 power sources are stacked together to show the overall power
    production.
    """
    trends = []
    revenues = []
    
    stack=False
    df = pd.read_csv('3_mo_weekly2.csv', sep='\t')
    df['Date_dt'] = df['Date'].astype('datetime64[ns]')
    df['google trends'] = df['google trends'] * df['Release'].apply(lambda x: len(re.sub('2020 Re-release', '', x).split(' ')))

    date = df['Date_dt'].iloc[-1]
    if df is None:
        return go.Figure()
    
    fig = go.Figure()
    movie_list = df[df['Date_dt'] == date].sort_values(by=['Weekly'], ascending=False)['Release'][0:20]
    for row in range(df.shape[0]):
        date = df['Date_dt'].iloc[-1]
        trend = df.iloc[row]['google trends']
        #if trend > 40:
        #    continue
        rev = df.iloc[row]['Weekly']
        #if trend > 1 and rev <1e6:
        #    continue
        if trend == 0:
            continue
        if rev == 0:
            continue
        trends.append(trend)
        revenues.append(rev)
        fig.add_trace(go.Scatter(x=[trend], y=[rev], mode='markers', name=df.iloc[row]["Release"],
                         line={'width': 2, 'color': c[row%4]},
                         stackgroup='stack' if stack else None))
    trends, revenues = pd.Series(trends), pd.Series(revenues)
    corr = trends.corr(revenues)
    
    title = f'Google Trends vs Weekly Gross: Correlation = {corr}'
    if stack:
        title += ' [Stacked]'
    fig.update_layout(template='plotly_dark',
                      title=title,
                      plot_bgcolor='#23272c',
                      paper_bgcolor='#23272c',
                      yaxis_title='Weekly Gross',
                      xaxis_title='Google Trend')
    return fig
static_stacked_trend_graph()

In [47]:
import plotly.graph_objects as go
import pandas as pd
from datetime import timedelta


COLORS = ['rgb(67,67,67)', 'rgb(115,115,115)', 'rgb(49,130,189)', 'rgb(189,189,189)']
c = ['red', 'blue', 'orange', 'white']
def static_stacked_trend_graph(stack=False):
    """
    Returns scatter line plot of all power sources and power load.
    If `stack` is `True`, the 4 power sources are stacked together to show the overall power
    production.
    """
    trends = []
    revenues = []
    
    stack=False
    df = pd.read_csv('2_mo_weekly.csv', sep='\t')
    df['Date_dt'] = df['Date'].astype('datetime64[ns]')
    date = df['Date_dt'].iloc[-1]
    if df is None:
        return go.Figure()
    
    fig = go.Figure()
    movie_list = df[df['Date_dt'] == date].sort_values(by=['Weekly'], ascending=False)['Release'][0:20]
    for row in range(df.shape[0]):
        date = df['Date_dt'].iloc[-1]
        trend = df.iloc[row]['Weekly']
        #if trend > 40:
        #    continue
        rev = df.iloc[row]['Week + 1']
        #if trend > 1 and rev <1e6:
        #    continue
        #if trend == 0:
        #    continue
        trends.append(trend)
        revenues.append(rev)
        fig.add_trace(go.Scatter(x=[trend], y=[rev], mode='markers', name=df.iloc[row]["Release"],
                         line={'width': 2, 'color': c[row%4]},
                         stackgroup='stack' if stack else None))
    trends, revenues = pd.Series(trends), pd.Series(revenues)
    corr = trends.corr(revenues)
    
    title = f'Weekly vs Week + 1: Correlation = {corr}'
    if stack:
        title += ' [Stacked]'
    fig.update_layout(template='plotly_dark',
                      title=title,
                      plot_bgcolor='#23272c',
                      paper_bgcolor='#23272c',
                      yaxis_title='Weekly Gross',
                      xaxis_title='Week + 1')
    return fig
static_stacked_trend_graph()

In [33]:
import plotly.graph_objects as go

df = pd.read_csv('3_mo_weekly.csv', sep='\t')
df['Date_dt'] = df['Date'].astype('datetime64[ns]')
df['google trends'] = df['google trends'] * df['Release'].apply(lambda x: len(re.sub('2020 Re-release', '', x).split(' ')))


for i in range(len(df['Date_dt'].unique())):
    date = df['Date_dt'].unique()[i]  #.iloc[-10]
    
    corr_list = []
    fig = go.Figure()
    sorted_df = df[df['Date_dt'] == date].sort_values(by=['Weekly'], ascending=False)[0:15][::-1]
    y = sorted_df['Release']
    x_rev = sorted_df['Weekly'] / np.sum(sorted_df['Weekly'])
    x_trend = sorted_df['google trends'] / np.sum(sorted_df['google trends'])
    trends, revenues = pd.Series(x_trend), pd.Series(x_rev)
    corr = trends.corr(revenues)
    print(corr)
    #plt.scatter(x_rev, x_trend)
    #plt.show()
    corr_list.append(corr)
    continue
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        y=y,
        x=x_rev,
        name='Gross',
        orientation='h',
        marker=dict(
            color='blue', #rgba(246, 78, 139, 0.6)',
            line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
        )
    ))
    fig.add_trace(go.Bar(
        y=y,
        x=x_trend,
        name='Google Trends',
        orientation='h',
        marker=dict(
            color='red', #rgba(58, 71, 80, 0.6)',
            line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
        )
    ))
    title = f'Normalized Gross and Google Trends for top 15 Movies: Correlation = {corr}'

    #fig.update_layout(barmode='stack')
    fig.update_layout(template='plotly_dark',
                      title=title,
                      plot_bgcolor='#23272c',
                      paper_bgcolor='#23272c',
                      yaxis_title='Movies',
                      xaxis_title='Normalized Quantities',barmode='stack')


    fig.show()


0.41037587135014253
0.7194935800258426
0.6506533154018306
0.4305436384157956
0.31896437197444805
-0.027364522535755727
-0.013952650934437682
0.09421258149106075
-0.11738711937268864
-0.1338400485409121
0.08540554010796937
0.8378120873181539
0.8702130302810439
0.859311248055649
0.7200875932162227
0.6805825730787766


In [38]:
import plotly.graph_objects as go
import numpy as np
import re

df = pd.read_csv('3_mo_weekly.csv', sep='\t')
df['Date_dt'] = df['Date'].astype('datetime64[ns]')
#df['google trends'] = df['google trends'] * df['Release'].apply(lambda x: len(re.sub('2020 Re-release', '', x).split(' ')))

date = df['Date_dt'].unique()[-1]#.iloc[-10]
print(date)

fig = go.Figure()
sorted_df = df[df['Date_dt'] == date].sort_values(by=['Weekly'], ascending=False)[0:15][::-1]
y = sorted_df['Release']
x_rev = sorted_df['Weekly'] / np.sum(sorted_df['Weekly'])
x_trend = sorted_df['google trends'] / np.sum(sorted_df['google trends'])
trends, revenues = pd.Series(x_trend), pd.Series(x_rev)
corr = trends.corr(revenues)

fig = go.Figure()
fig.add_trace(go.Bar(
    y=y,
    x=x_rev,
    name='Gross',
    orientation='h',
    marker=dict(
        color='blue', #rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=y,
    x=x_trend,
    name='Google Trends',
    orientation='h',
    marker=dict(
        color='red', #rgba(58, 71, 80, 0.6)',
        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
))
title = f'Normalized Gross and Google Trends for top 15 Movies: Correlation = {corr}'

#fig.update_layout(barmode='stack')
fig.update_layout(template='plotly_dark',
                  title=title,
                  plot_bgcolor='#23272c',
                  paper_bgcolor='#23272c',
                  yaxis_title='Movies',
                  xaxis_title='Normalized Quantities',barmode='stack')


#fig.show()

2020-12-07T00:00:00.000000000


In [65]:
import plotly.graph_objects as go

df = pd.read_csv('6_mo_weekly.csv', sep='\t')
df['Date_dt'] = df['Date'].astype('datetime64[ns]')

date = df['Date_dt'].unique()[5]#.iloc[-10]
print(date)

fig = go.Figure()
sorted_df = df[df['Date_dt'] == date].sort_values(by=['Weekly'], ascending=False)[0:15][::-1]
y = sorted_df['Release']
x_rev = sorted_df['Weekly'] / np.sum(sorted_df['Weekly'])
x_trend = sorted_df['google trends'] / np.sum(sorted_df['google trends'])
trends, revenues = pd.Series(x_trend), pd.Series(x_rev)
corr = trends.corr(revenues)

fig = go.Figure()
fig.add_trace(go.Bar(
    y=y,
    x=x_rev,
    name='Gross',
    orientation='h',
    marker=dict(
        color='blue', #rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=y,
    x=x_trend,
    name='Google Trends',
    orientation='h',
    marker=dict(
        color='red', #rgba(58, 71, 80, 0.6)',
        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
))
title = f'Normalized Gross and Google Trends for top 15 Movies: Correlation = {corr}'

#fig.update_layout(barmode='stack')
fig.update_layout(template='plotly_dark',
                  title=title,
                  plot_bgcolor='#23272c',
                  paper_bgcolor='#23272c',
                  yaxis_title='Movies',
                  xaxis_title='Normalized Quantities',barmode='stack')


#fig.show()

2020-06-22T00:00:00.000000000


In [6]:
date(2020,8,24)

datetime.date(2020, 8, 24)

In [12]:
import sched, time
from datetime import date, timedelta, datetime
import pandas as pd
from bs4 import BeautifulSoup
import requests
import gtab
import numpy as np
import datetime
import re


#DOWNLOAD_PERIOD = 604800

def add_target(df_weekly):
    '''Matches a row with the next weeks 'Weekly' columns which is target variable.  Performed in 
        this way instead of using merge to avoid SettingWithCopyWarning'''  
    df_temp = pd.read_csv('3_mo_weekly.csv', sep='\t')
    df_weekly = df_temp.append(df_weekly, ignore_index=True)
    df_weekly['Date'] = df_weekly['Date'].astype('datetime64[ns]')
    df_weekly['Y'] = df_weekly['Date'].apply(lambda x: x + timedelta(days=7))
    df_weekly['Week + 1'] = pd.Series(np.zeros(df_weekly.shape[0]))
    for movie in df_weekly['Release'].unique():
        for date in df_weekly[df_weekly['Release'] == movie]['Date']:
            df_weekly.loc[(df_weekly['Release'] == movie) & (df_weekly['Y'] == date), 'Week + 1'] = float(df_weekly.loc[(df_weekly['Release'] == movie) & (df_weekly['Date'] == date)]['Weekly'])
            df_weekly['Week + 1'].fillna(0, inplace=True)
    return df_weekly


def to_weekly(df):
    '''Combines daily into weekly data, divides by 7 where appropriate, removes data points with 'Weekly' == 0'''
        #convert date column into datetime object
    df['Date'] = df['Date'].astype('datetime64[ns]')
    #convert daily data to weekly
    df_weekly = df.groupby("Release").resample('W-Mon', label='left', closed = 'left', on='Date').sum().reset_index().sort_values(by='Date')
    df_weekly['Avg TD'] = df_weekly['TD'].apply(lambda x: x/7)
    df_weekly = df_weekly.drop('TD', axis=1)
    df_weekly['Avg YD'] = df_weekly['YD'].apply(lambda x: x/7)
    df_weekly = df_weekly.drop('YD', axis=1)
    df_weekly['Weekly'] = df_weekly['Daily']  
    df_weekly = df_weekly.drop('Daily', axis=1)
    df_weekly['Weekly %+-YD'] = df_weekly['%+-YD'] 
    df_weekly['Weekly %+-LW'] = df_weekly['%+-LW'] 
    df_weekly['Avg Theatre'] = df_weekly['Theatre'].apply(lambda x: x/7)
    df_weekly = df_weekly.drop('Theatre', axis=1)
    df_weekly['Avg per Theatre Avg'] = df_weekly['Avg'].apply(lambda x: x/7)
    df_weekly = df_weekly.drop('Avg', axis=1)
    df_weekly['Avg To Date'] = df_weekly['To Date'].apply(lambda x: x/7)
    df_weekly = df_weekly.drop('To Date', axis=1)
    df_weekly = df_weekly[df_weekly.Weekly != 0]
    
    return df_weekly
    

def edit_df(df):
    """Removes $,%,- and converts str to int or float"""
    df['Daily'] = df.Daily.apply(lambda x: x.strip('$'))
    df['%+-YD'] = df['%+-YD'].apply(lambda x: x.strip('%'))
    df['%+-LW'] = df['%+-LW'].apply(lambda x: x.strip('%'))
    df['Avg'] = df['Avg'].apply(lambda x: x.strip('$'))
    df['To Date'] = df['To Date'].apply(lambda x: x.strip('$'))
    df['Distributor'] = df.Distributor.apply(lambda x: x[0:-2])

    df = df.replace(',','', regex=True)
    df = df.replace('-', '0') #SHOUOLD BE NAN
    df = df.replace('<0.1', '0')
    
    df['Theatre'] = df['Theatre'].astype(float)
    df['Days'] = df['Days'].astype(float)
    df['Daily'] = df.Daily.astype(float)
    df['%+-YD'] = df['%+-YD'].astype(float)
    df['%+-LW'] = df['%+-LW'].astype(float)
    df['Avg'] = df['Avg'].astype(float)
    df['To Date'] = df['To Date'].astype(float)
    df['TD'] = df['TD'].astype(float)
    df['YD'] = df['YD'].astype(float)
    return df

def google_trends(df, d1, d2):
    """Pulls movie data from google trends and merges"""
    df.loc[:, 'google trends'] = 0
    for movie in df['Release'].unique():
        count = 0
        while True:
            if count == 5:
                break
            try:
                movie_search_term = re.sub('\:.*$', '', movie)
                movie_search_term = re.sub('2020 Re-release', '', movie_search_term)
                movie_search_term = movie_search_term + ' movie'

                t = gtab.GTAB();
                t.set_options(pytrends_config={"timeframe": f"{str(d1)} {str(d2)}"}); 
                query = t.new_query(movie_search_term);

                for date in query.index:
                    df.loc[df['Release'].eq(movie) & df['Date'].eq(date), 'google trends'] = query.loc[date, 'max_ratio']
            except:
                count += 1
                continue
            break
    return df

def get_data():
    '''Scrapes data from boxofficemojo with beautiful soup'''
    
    #d2 = datetime.date.today() - timedelta(days=2) #Change BACK!
    #d1 = d2 - timedelta(days=6)
    d2 = datetime.date(2020,12,27)
    d1 = datetime.date(2020,12,14)

    dd = [d1 + timedelta(days=x) for x in range((d2-d1).days + 1)][::-1]
    rows = 0
    df = pd.DataFrame(columns = ['Date','TD', 'YD', 'Release', 'Daily', '%+-YD', '%+-LW', 'Theatre', 'Avg', 'To Date', 'Days', 'Distributor'])

    for date in dd:
        count = 0
        while True:
            if count == 15:
                break
            try:
                source = requests.get('https://www.boxofficemojo.com/date/'+str(date)+'/').text
                soup = BeautifulSoup(source, 'lxml')
                table = soup.find('table')
                data = table.find_all('tr')

            except:
                count += 1
                continue
            break

        master_list = []

        for row in data:
            row_list = [date]
            try:
                for entry in row.find_all('td'):
                    if entry.text == 'false' or entry.text == 'true':
                        continue
                    row_list.append(entry.text)

                if len(row_list) == 12:
                    master_list.append(row_list)
            except:
                continue


        for i in range(len(master_list)):
            df.loc[rows] = master_list[i]
            rows += 1
            
    df = google_trends(df, d1, d2)
    df = edit_df(df) 
    df_weekly = to_weekly(df)
    df_weekly = add_target(df_weekly)

        
    df_weekly['Date'] = df_weekly['Date'].astype('datetime64[ns]')
    df_weekly.to_csv('test.csv', index=False, sep='\t') 
    print('DONE')
    
def is_it_tuesday():
    if datetime.datetime.today().weekday() == 4:
        get_data() 
        
is_it_tuesday()    
 
#Uncomment this code to use cron job    
#def main_loop(timeout=DOWNLOAD_PERIOD):
#    scheduler = sched.scheduler(time.time, time.sleep)
#    

#    def _worker():
#        try:
#            get_data()
#        except Exception as e:
#            logger.warning("main loop worker ignores exception and continues: {}".format(e))
#        scheduler.enter(timeout, 1, _worker)    # schedule the next event

#    scheduler.enter(0, 1, _worker)              # start the first event
#    scheduler.run(blocking=True)
#main_loop()

Using directory 'C:\Users\nclan\Anaconda3\lib\site-packages\gtab'
Active anchorbank changed to: google_anchorbank_geo=_timeframe=2019-01-01 2020-08-01.tsv

Using C:\Users\nclan\Anaconda3\lib\site-packages\gtab\output\google_anchorbanks\google_anchorbank_geo=_timeframe=2019-01-01 2020-08-01.tsv
New query 'The Croods movie'
New query calibrated!
Using directory 'C:\Users\nclan\Anaconda3\lib\site-packages\gtab'
Active anchorbank changed to: google_anchorbank_geo=_timeframe=2019-01-01 2020-08-01.tsv

Using C:\Users\nclan\Anaconda3\lib\site-packages\gtab\output\google_anchorbanks\google_anchorbank_geo=_timeframe=2019-01-01 2020-08-01.tsv
New query 'News of the World movie'
New query calibrated!
Using directory 'C:\Users\nclan\Anaconda3\lib\site-packages\gtab'
Active anchorbank changed to: google_anchorbank_geo=_timeframe=2019-01-01 2020-08-01.tsv

Using C:\Users\nclan\Anaconda3\lib\site-packages\gtab\output\google_anchorbanks\google_anchorbank_geo=_timeframe=2019-01-01 2020-08-01.tsv
New qu

New query calibrated!
Using directory 'C:\Users\nclan\Anaconda3\lib\site-packages\gtab'
Active anchorbank changed to: google_anchorbank_geo=_timeframe=2019-01-01 2020-08-01.tsv

Using C:\Users\nclan\Anaconda3\lib\site-packages\gtab\output\google_anchorbanks\google_anchorbank_geo=_timeframe=2019-01-01 2020-08-01.tsv
New query 'All My Life movie'
New query calibrated!
Using directory 'C:\Users\nclan\Anaconda3\lib\site-packages\gtab'
Active anchorbank changed to: google_anchorbank_geo=_timeframe=2019-01-01 2020-08-01.tsv

Using C:\Users\nclan\Anaconda3\lib\site-packages\gtab\output\google_anchorbanks\google_anchorbank_geo=_timeframe=2019-01-01 2020-08-01.tsv
New query 'Let Him Go movie'
New query calibrated!
Using directory 'C:\Users\nclan\Anaconda3\lib\site-packages\gtab'
Active anchorbank changed to: google_anchorbank_geo=_timeframe=2019-01-01 2020-08-01.tsv

Using C:\Users\nclan\Anaconda3\lib\site-packages\gtab\output\google_anchorbanks\google_anchorbank_geo=_timeframe=2019-01-01 2020-

Name: Weekly, dtype: float64
53    272658.0
Name: Weekly, dtype: float64
75    198932.0
Name: Weekly, dtype: float64
102    137208.0
Name: Weekly, dtype: float64
141    75090.0
Name: Weekly, dtype: float64
170    69201.0
Name: Weekly, dtype: float64
191    44521.0
Name: Weekly, dtype: float64
257    20611.0
Name: Weekly, dtype: float64
285    20332.0
Name: Weekly, dtype: float64
302    8092.0
Name: Weekly, dtype: float64
1    475800.0
Name: Weekly, dtype: float64
24    515111.0
Name: Weekly, dtype: float64
35    414661.0
Name: Weekly, dtype: float64
61    220681.0
Name: Weekly, dtype: float64
71    130798.0
Name: Weekly, dtype: float64
111    60467.0
Name: Weekly, dtype: float64
136    21712.0
Name: Weekly, dtype: float64
154    8533.0
Name: Weekly, dtype: float64
202    2273.0
Name: Weekly, dtype: float64
2    6480.0
Name: Weekly, dtype: float64
18    3225.0
Name: Weekly, dtype: float64
3    1125007.0
Name: Weekly, dtype: float64
25    1142565.0
Name: Weekly, dtype: float64
39    5397

Name: Weekly, dtype: float64
212    19452.0
Name: Weekly, dtype: float64
243    11810.0
Name: Weekly, dtype: float64
281    4176.0
Name: Weekly, dtype: float64
85    215675.0
Name: Weekly, dtype: float64
98    193730.0
Name: Weekly, dtype: float64
118    82645.0
Name: Weekly, dtype: float64
175    31940.0
Name: Weekly, dtype: float64
220    7440.0
Name: Weekly, dtype: float64
88    142694.0
Name: Weekly, dtype: float64
116    113701.0
Name: Weekly, dtype: float64
165    39664.0
Name: Weekly, dtype: float64
208    3855.0
Name: Weekly, dtype: float64
90    1925000.0
Name: Weekly, dtype: float64
137    1161000.0
Name: Weekly, dtype: float64
146    756000.0
Name: Weekly, dtype: float64
226    530000.0
Name: Weekly, dtype: float64
250    456000.0
Name: Weekly, dtype: float64
91    252664.0
Name: Weekly, dtype: float64
121    264534.0
Name: Weekly, dtype: float64
161    143954.0
Name: Weekly, dtype: float64
219    53946.0
Name: Weekly, dtype: float64
266    30698.0
Name: Weekly, dtype: float

310    3600355.0
Name: Weekly, dtype: float64
355    2053090.0
Name: Weekly, dtype: float64
394    1389310.0
Name: Weekly, dtype: float64
420    713195.0
Name: Weekly, dtype: float64
453    477130.0
Name: Weekly, dtype: float64
483    283500.0
Name: Weekly, dtype: float64
515    144380.0
Name: Weekly, dtype: float64
311    170000.0
Name: Weekly, dtype: float64
366    180000.0
Name: Weekly, dtype: float64
391    430000.0
Name: Weekly, dtype: float64
406    315000.0
Name: Weekly, dtype: float64
454    400000.0
Name: Weekly, dtype: float64
484    365000.0
Name: Weekly, dtype: float64
494    115000.0
Name: Weekly, dtype: float64
333    87552.0
Name: Weekly, dtype: float64
338    23383.0
Name: Weekly, dtype: float64
387    7250.0
Name: Weekly, dtype: float64
341    380428.0
Name: Weekly, dtype: float64
376    301009.0
Name: Weekly, dtype: float64
421    85227.0
Name: Weekly, dtype: float64
435    26861.0
Name: Weekly, dtype: float64
344    150497.0
Name: Weekly, dtype: float64
400    174157

In [5]:
import pandas as pd
df_weekly = pd.read_csv('3_mo_weekly.csv', sep='\t')
print(df_weekly.tail(3))

                         Release        Date  %+-YD  %+-LW   Days  \
459            After We Collided  2020-12-07   70.2 -441.0  295.0   
460          Wild Mountain Thyme  2020-12-07  -42.4    0.0    6.0   
461  Wonder Woman2020 Re-release  2020-12-07  -12.8    0.0    6.0   

     google trends     Avg TD     Avg YD    Weekly  Weekly %+-YD  \
459       2.600000  17.428571  14.857143     179.0          70.2   
460       0.000000   6.714286   4.428571   95360.0         -42.4   
461       2.673684   4.000000   2.571429  189000.0         -12.8   

     Weekly %+-LW  Avg Theatre  Avg per Theatre Avg   Avg To Date           Y  \
459        -441.0     4.285714             6.142857  2.045465e+06  2020-12-14   
460           0.0   192.857143            30.000000  2.947229e+04  2020-12-14   
461           0.0     0.000000             0.000000  5.642857e+04  2020-12-14   

     Week + 1  
459       0.0  
460       0.0  
461       0.0  


In [11]:
d2 = datetime.date(2020,12,27)
d2

datetime.date(2020, 12, 27)